In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import pickle 
import os 
import time 
import tensorflow as tf 

In [3]:
## Load the trained model and all the pickle files

model = tf.keras.models.load_model('/Users/rithul.v/Desktop/Study/Udemy/ANN_Projects/Trained_Model/Model_2025_01_21_23_23_02.keras')

2025-01-22 12:23:58.391978: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-22 12:23:58.392015: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-22 12:23:58.392023: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-22 12:23:58.392209: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-22 12:23:58.392232: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/rithul.v/Desktop/Study/Udemy/ANN_Projects/venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop',

In [4]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [5]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [8]:
onehot_encoder_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [9]:
#Example Input data

input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [10]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(
    geo_encoded, 
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)


/Users/rithul.v/Desktop/Study/Udemy/ANN_Projects/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [11]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_data = pd.DataFrame([input_data])

In [13]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [14]:
input_data = pd.concat([input_data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [15]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [16]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])

In [17]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
## Scaling the data 
input_scaled = scaler.transform(input_data)

In [19]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [20]:
## Predict Churn
prediction = model.predict(input_scaled)
print(prediction)

2025-01-22 12:25:11.320895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step
[[0.0557608]]


In [21]:
prediction_prob = prediction[0][0]

In [22]:
prediction_prob

0.055760805

In [23]:
if prediction_prob > 0.5:
    print("The Customer is likerly to churn.")
else:
    print("The customerr is not likely to churn.")

The customerr is not likely to churn.
